# Project 2 - Part 2

In [1]:
pip install -r requirements.txt


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Supervised Model for Transfer

## Representation Learning Model

## Visualising Learned Representations

## Finetuning Strategies